## IMPORTING REQUIRED LIBRARIES

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import vtk

## FETCHING AND READING THE VTI DATA

In [2]:
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("mixture.vti")
reader.Update()
data = reader.GetOutput()
scalar_f = data.GetPointData().GetScalars()
dims = data.GetDimensions()

## CREATING MESH USING X, Y AND Z COORDINATE DATA AND CONVERTING IT TO PLOTLY DATATYPE

In [3]:
origin = data.GetOrigin()
spacing = data.GetSpacing()
x = np.linspace(origin[0], origin[0] + spacing[0] * (dims[0] - 1), dims[0])
y = np.linspace(origin[1], origin[1] + spacing[1] * (dims[1] - 1), dims[1])
z = np.linspace(origin[2], origin[2] + spacing[2] * (dims[2] - 1), dims[2])
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
scalar_values = np.array(scalar_f).reshape(dims, order='F')

## CREATING A FIGURE WIDGET FOR THE ISOSURFACE WITH INITIAL ISOVALUE AS 0.0

In [4]:
fig_iso = go.FigureWidget()
isosurface = fig_iso.add_trace(go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=scalar_values.flatten(),
    isomin=0.0,
    isomax=0.0,
    cmin=np.min(scalar_values),
    cmax=np.max(scalar_values),
    colorscale='plasma',
    showscale=False
))
fig_iso.update_layout(scene=dict(
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False),
    zaxis=dict(showticklabels=False)
))

FigureWidget({
    'data': [{'cmax': 0.43280163,
              'cmin': -0.99355406,
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'isomax': 0.0,
              'isomin': 0.0,
              'showscale': False,
              'type': 'isosurface',
              'uid': '9d3e6c93-f3fe-422b-a65f-6457a4a272cb',
              'value': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.9426848 , -0.8584828 ,
                              -0.72851294], dtype=float32),
              'x': array([7.45000000e-05, 7.45000000e-05, 7.45000000e-05, ..., 1.48999926e+0

## CREATING THE INITIAL HISTOGRAM FOR COMPLETE DATA

In [5]:
subset_data = scalar_values.flatten()
fig_histo = go.FigureWidget()
histogram = fig_histo.add_trace(go.Histogram(
    x=subset_data,
    marker=dict(color='blue'),
    opacity=0.7,
    nbinsx=30
))
histogram.update_layout(
            xaxis_title='Vortex Scalar Values',
            yaxis_title='Frequency'
        )

FigureWidget({
    'data': [{'marker': {'color': 'blue'},
              'nbinsx': 30,
              'opacity': 0.7,
              'type': 'histogram',
              'uid': 'a8adf5db-71e5-482f-969a-4a5cc1d52657',
              'x': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.9426848 , -0.8584828 ,
                          -0.72851294], dtype=float32)}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'Vortex Scalar Values'}},
               'yaxis': {'title': {'text': 'Frequency'}}}
})

## CREATING SLIDER AND BUTTON WITH THE FUNCTION THAT CHANGES ISOSURAFCE AND HISTOGRAM

In [6]:
flag = 0
def func_for_update(isovalue):
    global flag
    isosurface.data[0].isomin = isovalue
    isosurface.data[0].isomax = isovalue
    if(round(isovalue,2)==0.0 and flag<=1):
        subset_data = scalar_values.flatten()
        flag += 1
    elif(flag>1):
        subset_data = scalar_values[(scalar_values >= isovalue - 0.25) & (scalar_values <= isovalue + 0.25)].flatten()
    with histogram.batch_update():
        histogram.data[0].x = subset_data
    isosurface.update_traces()

widget1 = widgets.FloatSlider(value=0.0, min=np.min(scalar_values), max=np.max(scalar_values), step=0.01, 
                              description="Isovalue:", layout=widgets.Layout(width='50%'), continous_update=False)
output = widgets.interactive_output(func_for_update, {'isovalue':widget1})

def reset_func(isovalue):
    global flag
    widget1.value=0.0
    flag = 0
    func_for_update(0.0)

button = widgets.Button(description="Reset")
button.on_click(reset_func)

## DISPLAYING ALL THE COMPONENTS TOGETHER

In [7]:
final_display = widgets.VBox([widgets.HBox([widget1, button]), widgets.HBox([isosurface, histogram])])
display(final_display)